In [5]:
# 防止过拟合的方法：增加数据集、正则化项、Dropout
# 如果连接一个神经元的所有权值为0 可以认为这个神经元不存在 减小了网络的复杂程度

# Dropout：训练的时候选择部分神经元训练，测试的时候激活所有神经元
# Dropout：缺点：训练速度变慢
#          优点：训练集准确性 和 测试集准确性 曲线很接近
# 何时使用：当训练数据集很小，要训练的网络很大时，训练时不用dropout过拟合现象会非常明显

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)                                             # 把标签转换为只有0 1的热键形式

# 每个批次的大小
batch_size = 40
n_batch = mnist.train.num_examples//batch_size                                                         # 这句看不懂 一共有多少个批次 整除

# 给输入数据预留位置
x = tf.placeholder(tf.float32, shape=[None,784])
y = tf.placeholder(tf.float32, shape=[None,10])
keep_prob = tf.placeholder(tf.float32)

# 创建网络
W1 = tf.Variable(tf.truncated_normal([784,2000],stddev=0.1))                                                                # 784个2000列
b1 = tf.Variable(tf.zeros([1,2000])+0.1)                                                                                     # 2000个1列
Wx_plus_b1 = tf.matmul(x,W1) + b1
L1 = tf.nn.tanh(Wx_plus_b1)
L1_drop = tf.nn.dropout(L1,keep_prob)  # 某一层神经元的输出 有百分之多少的神经元工作

#W2 = tf.Variable(tf.truncated_normal([2000,2000],stddev=0.1))
#b2 = tf.Variable(tf.zeros([1,2000])+0.1)
#Wx_plus_b2 = tf.matmul(L1_drop,W2) + b2
#L2 = tf.nn.tanh(Wx_plus_b2)
#L2_drop = tf.nn.dropout(L2,keep_prob)

W3 = tf.Variable(tf.truncated_normal([2000,1000],stddev=0.1))
b3 = tf.Variable(tf.zeros([1,1000])+0.1)
Wx_plus_b3 = tf.matmul(L2_drop,W3) + b3
L3 = tf.nn.tanh(Wx_plus_b3)
L3_drop = tf.nn.dropout(L3,keep_prob)

W4 = tf.Variable(tf.truncated_normal([1000,10],stddev=0.1))
b4 = tf.Variable(tf.zeros([1,10])+0.1)
Wx_plus_b4 = tf.matmul(L3_drop,W4) + b4

# 预测值
# prediction = tf.nn.softmax(Wx_plus_b4)
# loss = tf.reduce_mean(tf.square(y-prediction))                                                            # 二次代价函数  和 softmax 配
# prediction = tf.nn.sigmoid(Wx_plus_b4)
# loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=Wx_plus_b4))# softmax 交叉熵  需要sigmoid激活 标签 预测值
prediction = tf.nn.softmax(Wx_plus_b4)
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=Wx_plus_b4))

train = tf.train.GradientDescentOptimizer(0.1).minimize(loss)

# 初始化变量
init = tf.global_variables_initializer()
# 定义求准确率的方法 测试一下模型的准确率
# 这个y是标签 y的位置是数字答案 这里其实就是用预测值（prediction刚好由softmax得概率）与标签所得答案比较 得到 布尔型列表 
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(prediction,1))     # 返回 Ture False 求出最大的数在的位置 看它俩返回的位置一样不
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))                                        # cast是强制类型转换 然后 求均值


Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [5]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(5):                                                                                              # 两层循环是训练
        for pic in range(n_batch):                                # 每次训练 是从第n个批次里取1个数据 循环结束 所有批次中的数据被训练一次
            pic_data,pic_tag = mnist.train.next_batch(batch_size)   # 从train集中选取batch_size个训练数据 next_batch(我觉得这是bagging 错)
            sess.run(train,feed_dict={x:pic_data,y:pic_tag,keep_prob:0.8})                              # 获取一个训练数据 这儿就训练一次   
        # 每一次大循环 看一下准确率
        if epoch % 1 == 0:
            train_accuracy_rate = sess.run(accuracy,feed_dict={x:mnist.train.images,y:mnist.train.labels,keep_prob:1.0})   # 训练集正确率
            test_accuracy_rate = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})      # 测试集正确率
            print('第' + str(epoch+1) + '次，train准确率 ' + str(train_accuracy_rate) + '  test准确率 ' + str(test_accuracy_rate))

InternalError: Blas GEMM launch failed : a.shape=(40, 784), b.shape=(784, 2000), m=40, n=2000, k=784
	 [[node MatMul_4 (defined at <ipython-input-4-02b428cbfbf7>:16) ]]

Caused by op 'MatMul_4', defined at:
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\tornado\platform\asyncio.py", line 127, in start
    self.asyncio_loop.run_forever()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\asyncio\bas

In [ ]:
prediction = tf.nn.softmax(Wx_plus_b4)   # keep_prob:1.0
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=Wx_plus_b4))

第1次，train准确率 0.96378183
第1次， test准确率 0.9536
第2次，train准确率 0.9799455
第2次， test准确率 0.9634
第3次，train准确率 0.9934
第3次， test准确率 0.9735
第4次，train准确率 0.99883634
第4次， test准确率 0.9773
第5次，train准确率 1.0
第5次， test准确率 0.9799

In [ ]:
prediction = tf.nn.softmax(Wx_plus_b4)   # keep_prob:1.0
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

第1次，train准确率 0.9602727
第1次， test准确率 0.9491
第2次，train准确率 0.9775818
第2次， test准确率 0.9618
第3次，train准确率 0.9843636
第3次， test准确率 0.9664
第4次，train准确率 0.98736364
第4次， test准确率 0.9679
第5次，train准确率 0.98983634
第5次， test准确率 0.9696

In [ ]:
prediction = tf.nn.sigmoid(Wx_plus_b4)   # keep_prob:1.0
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

第1次，train准确率 0.95934546
第1次， test准确率 0.948
第2次，train准确率 0.97376364
第2次， test准确率 0.9589
第3次，train准确率 0.98032725
第3次， test准确率 0.963
第4次，train准确率 0.9844364
第4次， test准确率 0.9672
第5次，train准确率 0.9866545
第5次， test准确率 0.9674

In [ ]:
prediction = tf.nn.sigmoid(Wx_plus_b4)   # keep_prob:1.0
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=Wx_plus_b4))

第1次，train准确率 0.97049093
第1次， test准确率 0.9584
第2次，train准确率 0.9866
第2次， test准确率 0.9682
第3次，train准确率 0.99347275
第3次， test准确率 0.9736
第4次，train准确率 0.99945456
第4次， test准确率 0.9789
第5次，train准确率 1.0
第5次， test准确率 0.9795